In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20,256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2,20)
net(X)

tensor([[ 0.0736, -0.1750, -0.0318, -0.1629,  0.0723,  0.1651,  0.1221, -0.1885,
          0.0573, -0.1691],
        [ 0.0452, -0.1218,  0.0254, -0.1957,  0.0567,  0.1638,  0.1955, -0.2173,
          0.0465, -0.0681]], grad_fn=<AddmmBackward0>)

In [2]:
class MLP(nn.Module):

    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.out = nn.Linear(256,10)

    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP()
net(X)

tensor([[-0.0085, -0.0068,  0.0341,  0.0377,  0.3580,  0.0129,  0.0718,  0.0444,
          0.1430, -0.0527],
        [-0.1870, -0.0270,  0.0091,  0.0851,  0.3531,  0.0878,  0.1021,  0.0913,
          0.0119, -0.1631]], grad_fn=<AddmmBackward0>)

In [10]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [11]:
net = MySequential(nn.Linear(20,256), nn.ReLU(), nn.Linear(256,10))
net(X)

tensor([[ 0.0595,  0.2201,  0.0003,  0.1580,  0.0596,  0.0369,  0.0597,  0.0385,
         -0.2111, -0.0623],
        [ 0.2035,  0.2493,  0.0655,  0.1522,  0.0292,  0.1516,  0.0386, -0.1266,
         -0.0799, -0.0542]], grad_fn=<AddmmBackward0>)

In [12]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20,20), requires_grad=False)
        self.linear = nn.Linear(20,20)

    def forward(self,X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [13]:
net = FixedHiddenMLP()
net(X)

tensor(-0.0939, grad_fn=<SumBackward0>)

In [14]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64), nn.ReLU(),
        nn.Linear(64,32), nn.ReLU())
        self.linear = nn.Linear(32,16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16,20), FixedHiddenMLP())
chimera(X)

tensor(-0.2298, grad_fn=<SumBackward0>)